In [1]:
import pandas as pd
import numpy as np

In [2]:
class HMM(object):
    """
    Finite state space hidden markov model.
    """
    
    # Problem 1
    def __init__(self):
        """
        Initialize an HMM with parameters A, B, and pi, initialized as 
        None objects.
        """
        self.A = None #initialize this as None
        self.B = None #initialize this as None
        self.pi = None #initialize this as None
    
    
    # Problem 2
    def _forward(self, z):
        """
        Compute the scaled forward probability matrix and scaling factors.
        
        Parameters
        ----------
        z : ndarray of shape (T,) 
            The sequence of observations
        
        Returns
        -------
        alpha : ndarray of shape (T, n)
            The scaled forward probability matrix
        
        c : ndarray of shape (T,)
            The scaling factors c = [c_0, c_1, ..., c_{T-1}]
        """
        alpha, c = np.zeros((len(z),len(self.pi))), np.zeros(len(z)) #initialize this as a zero array
        alpha[0] = self.pi * self.B[z[0]] #set the first row
        c[0] = 1/(sum(alpha[0])) #and the first entry
        alpha[0] *= c[0] #scale it
        for t in range(1,len(alpha)): #for the rest of the rows
            alpha[t] = np.array([sum([alpha[t-1][j]*self.A[i,j]*self.B[z[t],i] for j in range(len(self.pi))]) for i in range(len(self.pi))]) #do this
            c[t] = 1/(sum(alpha[t])) #then get c
            alpha[t] *= c[t] #and scale
        return alpha, c #and return the result
    
    
    # Problem 3
    def _backward(self, z, c):
        """
        Compute the scaled backward probability matrix.
        
        Parameters
        ----------
        z : ndarray of shape (T,) 
            The sequence of observations
        c : ndarray of shape (T,)
            The scaling factors from the forward pass

        Returns
        -------
        beta : ndarray of shape (T, n)
            The scaled backward probability matrix
        """
        T = len(z) #for later use
        beta = np.zeros((len(z),len(self.pi))) #initialize beta as a zero matrix
        beta[T-1] = c[T-1] #make the last row
        for t in range(T-2,-1,-1): #for the rest of the rows
            beta[t] = np.array([sum([beta[t+1][j]*self.A[j,i]*self.B[z[t+1],j] for j in range(len(self.pi))]) for i in range(len(self.pi))]) #compute beta like this
            beta[t] *= c[t] #then multiply by c
        return beta #and return beta
    
    # Problem 4
    def _xi(self, z, alpha, beta, c):
        """
        Compute the xi and gamma probabilities.

        Parameters
        ----------
        z : ndarray of shape (T,)
            The observation sequence
        alpha : ndarray of shape (T, n)
            The scaled forward probability matrix from the forward pass
        beta : ndarray of shape (T, n)
            The scaled backward probability matrix from the backward pass
        c : ndarray of shape (T,)
            The scaling factors from the forward pass

        Returns
        -------
        xi : ndarray of shape (T-1, n, n)
            The xi probability array
        gamma : ndarray of shape (T, n)
            The gamma probability array
        """
        xi = np.zeros(((len(z)-1),len(alpha[0]),len(alpha[0]))) #initialize this as a zero array
        for t in range(len(z)-1): #for each t
            for i in range(len(alpha[0])): #for each i
                for j in range(len(alpha[0])): #for each j
                    xi[t,i,j] = alpha[t,i]*self.A[j,i]*self.B[z[t+1],j]*beta[t+1,j] #compute xi entry
        gamma = alpha*beta/c[:,None] #and get gamma
        return xi, gamma #return them both
            
    
    # Problem 5
    def _estimate(self, z, xi, gamma):
        """
        Estimate better parameter values and update self.pi, self.A, and
        self.B in place.

        Parameters
        ----------
        z : ndarray of shape (T,)
            The observation sequence
        xi : ndarray of shape (T-1, n, n)
            The xi probability array
        gamma : ndarray of shape (T, n)
            The gamma probability array
        """
        self.pi = gamma[0] #this is pi
        self.A = (sum(xi[:])/sum(gamma[:-1])[:,None]).T #this is A
        self.B = np.array([sum(gamma[z==i]) for i in set(z)])/sum(gamma) #this is B

    
    # Problem 6
    def fit(self, z, pi, A, B, max_iter=100, tol=1e-5):
        """
        Fit the model parameters to a given observation sequence.

        Parameters
        ----------
        z : ndarray of shape (T,)
            Observation sequence on which to train the model.
        pi : Stochastic ndarray of shape (n,)
            Initial state distribution
        A : Stochastic ndarray of shape (n, n)
            Initial state transition matrix
        B : Stochastic ndarray of shape (m, n)
            Initial state observation matrix
        max_iter : int
            The maximum number of iterations to take
        tol : float
            The convergence threshold for change in log-probability
        """
        # initialize self.pi, self.A, self.B
        self.pi, self.A, self.B = pi, A, B #save these as attributes
        # run the iteration
        old_logprob = np.inf #old prob should be big
        for i in range(max_iter): #do this as most max_iter times
            alpha, c = self._forward(z) #get alpha and c from forward
            beta = self._backward(z, c) #and beta from backward
            xi, gamma = self._xi(z, alpha, beta, c) #then use this method to get xi and gamma
            self._estimate(z, xi, gamma) #do the estimation
            new_logprob = -sum(np.log(c)) #compute the new logprob
            if abs(new_logprob-old_logprob) < tol: #see if we are within the tol
                break #if so be done
            old_logprob = new_logprob #otherwise iterate


A first attempt at the problem. Here we use the exact sales amount for that day as the observation and searched for 7 hidden states (hoping to hit weekdays). We did this over a single medicine (N02BE).

In [3]:
data = pd.read_csv('salesdaily.csv')
symbols = sorted((data.N02BE.dropna().unique().astype(float)*10))
obs = ((data.N02BE.dropna().astype(float)*10)).to_numpy()
state_index = {symbols[i]: i for i in range(len(symbols))}
obs_index = np.array([state_index[obs[i]] for i in range(len(obs))])
h = HMM() #initialize the model
N = 7 #set N
M = len(set(obs_index)) #set M
pi = np.random.dirichlet(np.ones(N)) #initialize pi
A = np.random.dirichlet(np.ones(N), size=N).T #initialize A
B = np.random.dirichlet(np.ones(M), size=N).T #initialize B
h.fit(obs_index,pi,A,B,max_iter=150) #fit the model

#display results
for i in range(len(h.B)):
    print(u"{0}, {1:0.4f}, {2:0.4f}, {3:0.4f}, {4:0.4f}, {5:0.4f}, {6:0.4f}, {7:0.4f}"
        .format(symbols[i]/10, h.B[i,0], h.B[i,1], h.B[i,2], h.B[i,3], h.B[i,4], h.B[i,5], h.B[i,6]))  

0.0, 0.0088, 0.0134, 0.0250, 0.0041, 0.0363, 0.0000, 0.0000
1.2, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0037, 0.0000
2.0, 0.0000, 0.0000, 0.0065, 0.0000, 0.0000, 0.0000, 0.0000
3.0, 0.0000, 0.0019, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000
6.0, 0.0029, 0.0019, 0.0032, 0.0000, 0.0000, 0.0000, 0.0000
6.2, 0.0000, 0.0000, 0.0000, 0.0035, 0.0000, 0.0000, 0.0000
6.5, 0.0000, 0.0019, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000
6.6, 0.0000, 0.0019, 0.0000, 0.0035, 0.0000, 0.0000, 0.0000
6.7, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0055
6.8, 0.0000, 0.0000, 0.0033, 0.0000, 0.0000, 0.0000, 0.0000
7.0, 0.0029, 0.0058, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000
7.25, 0.0000, 0.0000, 0.0033, 0.0000, 0.0000, 0.0000, 0.0000
7.8, 0.0000, 0.0019, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000
8.0, 0.0031, 0.0079, 0.0000, 0.0236, 0.0000, 0.0041, 0.0000
8.062, 0.0000, 0.0000, 0.0000, 0.0035, 0.0000, 0.0000, 0.0000
8.4, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0037, 0.0000
8.75, 0.0000, 0.0000, 0.0000, 0.0035,

This led to poor results as there is no ordering on states in HMM in the discrete case. We decided to try another approach.

We wanted our observation states to include all medicines, so we instead made our states multidimensional binary, where a 0 represents below average sales for a day and a 1 represent above average sales. Thus we can include information from all of our medicines in a single state. As our solver from the lab only took in integer values, we convert this multidimenional binary to base 10. For example, state [1,1,0,0,0,1,0,1] is passed in as 163.

In [4]:
#HMM guessing weekdays
data = pd.read_csv('salesdaily.csv')
means = data[['M01AB','M01AE','N02BA','N02BE','N05B','N05C','R03','R06']].mean()
#make high_low, which is 1 at the i,j spot if medicine i had above average sales on day j, and is zero otherwise.
high_low = data[['M01AB','M01AE','N02BA','N02BE','N05B','N05C','R03','R06']].where(data[['M01AB','M01AE','N02BA','N02BE','N05B','N05C','R03','R06']] >= means,0).where(data[['M01AB','M01AE','N02BA','N02BE','N05B','N05C','R03','R06']] < means,1).astype(int).to_numpy()
obs = high_low@np.array([1,2,4,8,16,32,64,128]) #convert to binary
obs = np.append(obs, [172]) #this state is never found, include it arbitrarily

symbols = range(256)
h = HMM() #initialize the model
N = 7 #set N
M = 256 #set M
pi = np.random.dirichlet(np.ones(N)) #initialize pi
A = np.random.dirichlet(np.ones(N), size=N).T #initialize A
B = np.random.dirichlet(np.ones(M), size=N).T #initialize B
h.fit(obs,pi,A,B,max_iter=150) #fit the model

#display results
for i in range(len(h.B)):
    print(u"{0}, {1:0.4f}, {2:0.4f}, {3:0.4f}, {4:0.4f}, {5:0.4f}, {6:0.4f}, {7:0.4f}"
        .format(symbols[i], h.B[i,0], h.B[i,1], h.B[i,2], h.B[i,3], h.B[i,4], h.B[i,5], h.B[i,6]))  

0, 0.0154, 0.0218, 0.0244, 0.1207, 0.0124, 0.0647, 0.0000
1, 0.0037, 0.0193, 0.0000, 0.0039, 0.0168, 0.0000, 0.0000
2, 0.0036, 0.0216, 0.0144, 0.0054, 0.0000, 0.0000, 0.0063
3, 0.0000, 0.0340, 0.0000, 0.0004, 0.0000, 0.0058, 0.0124
4, 0.0000, 0.0000, 0.0000, 0.0217, 0.0264, 0.0000, 0.0000
5, 0.0000, 0.0071, 0.0000, 0.0000, 0.0000, 0.0000, 0.0208
6, 0.0038, 0.0095, 0.0193, 0.0033, 0.0023, 0.0000, 0.0037
7, 0.0000, 0.0039, 0.0000, 0.0045, 0.0141, 0.0000, 0.0177
8, 0.0076, 0.0000, 0.0165, 0.0068, 0.0090, 0.0000, 0.0000
9, 0.0038, 0.0000, 0.0088, 0.0000, 0.0042, 0.0146, 0.0000
10, 0.0000, 0.0000, 0.0000, 0.0000, 0.0356, 0.0000, 0.0000
11, 0.0341, 0.0000, 0.0089, 0.0000, 0.0000, 0.0000, 0.0000
12, 0.0000, 0.0000, 0.0000, 0.0000, 0.0016, 0.0414, 0.0000
13, 0.0000, 0.0000, 0.0000, 0.0000, 0.0304, 0.0147, 0.0000
14, 0.0000, 0.0000, 0.0000, 0.0051, 0.0154, 0.0254, 0.0088
15, 0.0038, 0.0000, 0.0164, 0.0000, 0.0097, 0.0000, 0.0035
16, 0.0080, 0.0094, 0.0795, 0.0087, 0.0000, 0.0000, 0.0065
17, 0.0

In [5]:
#Printing the confusion matrix
guess_dict = dict()
for i in range(len(h.B)):
    guess_dict[symbols[i]] =np.argmax([h.B[i,0], h.B[i,1], h.B[i,2], h.B[i,3], h.B[i,4], h.B[i,5], h.B[i,6]])  
acc = np.zeros((7,7)) #will be the confusion matrix
daytoindex = {'Sunday':0,'Monday':1,'Tuesday':2,'Wednesday':3,'Thursday':4,'Friday':5,'Saturday':6}
for i in range(len(obs) - 1):
    acc[daytoindex[data['Weekday Name'][i]],guess_dict[obs[i]]] += 1
print(acc)

[[32. 35. 47. 63. 39. 44. 41.]
 [51. 49. 56. 26. 47. 27. 45.]
 [44. 65. 41. 39. 43. 37. 32.]
 [41. 43. 48. 38. 56. 35. 39.]
 [32. 45. 47. 44. 43. 43. 47.]
 [48. 49. 39. 38. 60. 31. 36.]
 [38. 41. 42. 45. 42. 42. 51.]]


There is hardly a strong indication of knowledge of one day over another.

In [6]:
#HMM guessing R06
data = pd.read_csv('salesdaily.csv')
means = data[['M01AB','M01AE','N02BA','N02BE','N05B','N05C','R03']].mean()
#make high_low, which is 1 at the i,j spot if medicine i had above average sales on day j, and is zero otherwise.
high_low = data[['M01AB','M01AE','N02BA','N02BE','N05B','N05C','R03']].where(data[['M01AB','M01AE','N02BA','N02BE','N05B','N05C','R03']] >= means,0).where(data[['M01AB','M01AE','N02BA','N02BE','N05B','N05C','R03']] < means,1).astype(int).to_numpy()
obs = high_low@np.array([1,2,4,8,16,32,64])

symbols = range(256)
h = HMM() #initialize the model
N = 2 #set N
M = len(set(obs)) #set M
pi = np.random.dirichlet(np.ones(N)) #initialize pi
A = np.random.dirichlet(np.ones(N), size=N).T #initialize A
B = np.random.dirichlet(np.ones(M), size=N).T #initialize B
h.fit(obs,pi,A,B,max_iter=150) #fit the model

guess_dict = dict()
for i in range(len(h.B)):
    guess_dict[symbols[i]] = np.argmax([h.B[i,0], h.B[i,1]])  
acc = np.zeros((2,2)) #will be the confusion matrix
R_means = data[['R06']].mean() #get the mean
R_high_low = data[['R06']].where(data[['R06']] >= R_means,0).where(data[['R06']] < R_means,1).astype(int).to_numpy().T[0] #and use it to make the confusion matrix

for i in range(len(obs)):
    acc[R_high_low[i],guess_dict[obs[i]]] += 1
print(acc)

[[538. 592.]
 [444. 532.]]


The model is not able to predict whether R06 will have above or below average sales on a given day.

In [7]:
#HMM guessing R03
data = pd.read_csv('salesdaily.csv')
means = data[['M01AB','M01AE','N02BA','N02BE','N05B','N05C','R06']].mean()
#make high_low, which is 1 at the i,j spot if medicine i had above average sales on day j, and is zero otherwise.
high_low = data[['M01AB','M01AE','N02BA','N02BE','N05B','N05C','R06']].where(data[['M01AB','M01AE','N02BA','N02BE','N05B','N05C','R06']] >= means,0).where(data[['M01AB','M01AE','N02BA','N02BE','N05B','N05C','R06']] < means,1).astype(int).to_numpy()
obs = high_low@np.array([1,2,4,8,16,32,64])

symbols = range(256)
h = HMM() #initialize the model
N = 2 #set N
M = len(set(obs)) #set M
pi = np.random.dirichlet(np.ones(N)) #initialize pi
A = np.random.dirichlet(np.ones(N), size=N).T #initialize A
B = np.random.dirichlet(np.ones(M), size=N).T #initialize B
h.fit(obs,pi,A,B,max_iter=150) #fit the model

guess_dict = dict()
for i in range(len(h.B)):
    guess_dict[symbols[i]] = np.argmax([h.B[i,0], h.B[i,1]])  
acc = np.zeros((2,2)) #will be the confusion matrix
R_means = data[['R03']].mean() #get the mean
R_high_low = data[['R03']].where(data[['R03']] >= R_means,0).where(data[['R03']] < R_means,1).astype(int).to_numpy().T[0] #and use it to make the confusion matrix

for i in range(len(obs)):
    acc[R_high_low[i],guess_dict[obs[i]]] += 1
print(acc)

[[520. 792.]
 [427. 367.]]


The model is not able to predict whether R03 will have above or below average sales on a given day.

We hoped to do this same analysis with monthly data (as a model should be able to predict months much better than days), but the data was not long enough to use HMM and ARIMA was working quite well, so we decided not to venture into CDHMM.